# Imports

In [1]:
import import_ipynb
from create_db_connection_strings import create_connection_string, create_sql_alchemy_engine
from read_env_file import SRVR, DTBS, USRNM, PSSWRD
from execute_sql import execute_sql_statement
import os
import pandas as pd

localhost master olist_user olist2025


In [2]:
# Current path
print(os.getcwd())

C:\Users\PC\Downloads\Project_OLIST - Copy\NOTEBOOKS


In [3]:
# Path of the dataset
csv_path = "../OLIST_DATASET/"

In [ ]:
# Get all of the csv file names
print(os.listdir(csv_path)) 
all_csv = os.listdir(csv_path)

In [5]:
# 4.1 Data Cleaning and Transformation

# 4.1.1 Load all CSV files into a Jupyter Notebook using Pandas
# 4.1.2 Clean the datasets:
# 4.1.2.2 Remove any duplicate rows

# This method reads the csv files and also drops the duplicates
def pandas_read_unique_csv(base_csv_path: str, file: str):
    print(f"The file path is: {base_csv_path}{file}")
    df = pd.read_csv(f"{base_csv_path}{file}")
    df_no_duplicates = df.drop_duplicates()
    return df_no_duplicates

In [6]:
# This method programatically creates variables and assigns them the dataframe created from a csv file
def create_dataframes_programatically(variable: str,value: str,base_path: str):
    var_name = variable
    globals()[var_name] = pandas_read_unique_csv(base_path,value)

In [ ]:
# This iterates through all of the csv files, creates the variables and the respective dataframes
for csv_file in all_csv:
    dtfrm = f"df_{csv_file.split('.')[0]}"
    print(f"The variable to be created is: {dtfrm}")
    create_dataframes_programatically(dtfrm,csv_file,csv_path)

In [ ]:
for csv_file in all_csv:
    var_name = f"df_{csv_file.split('.')[0]}"
    print(f"\nHead of: {var_name}")
    print(globals()[var_name].head())

In [9]:
# This creates a connection string based on the enviroment variables
arg_conn = create_connection_string(SRVR,DTBS,USRNM,PSSWRD)

In [10]:
# This creates the sqlalchemy engine from the connection
arg_eng = create_sql_alchemy_engine(arg_conn)

In [11]:
sql_stat = "CREATE DATABASE OLIST;"

In [12]:
rows = execute_sql_statement(arg_eng,sql_stat)

In [13]:
arg_conn_olist = create_connection_string(SRVR,"OLIST",USRNM,PSSWRD)

In [14]:
arg_eng_olist = create_sql_alchemy_engine(arg_conn_olist)

In [ ]:
# 4.1 Data Cleaning and Transformation

# 4.1.1 Load all CSV files into a Jupyter Notebook using Pandas
# 4.1.2 Clean the datasets:
# 4.1.2.1 Handle missing values by filling them with appropriate data or dropping rows/columns.

# olist_products
### fill null values
df_olist_products_dataset['product_category_name'].fillna('desconhecida', inplace=True)

df_olist_products_dataset['product_name_lenght'] = df_olist_products_dataset['product_name_lenght'].fillna(-1)

df_olist_products_dataset['product_description_lenght'] = df_olist_products_dataset['product_description_lenght'].fillna(-1)

df_olist_products_dataset['product_photos_qty'] = df_olist_products_dataset['product_description_lenght'].fillna(-1)

df_olist_products_dataset.to_sql("olist_products", con=arg_eng_olist, if_exists="replace", index=False)


In [16]:
# olist_sellers
df_olist_sellers_dataset.to_sql("olist_sellers", con=arg_eng_olist, if_exists="replace", index=False)

475

In [17]:
# product_category_name_translation
# Add a new category that was used to fill null values
df_product_category_name_translation.loc[len(df_product_category_name_translation)] = ['desconhecida', 'unknown']

df_product_category_name_translation.to_sql("product_category_name_translation", con=arg_eng_olist, if_exists="replace", index=False)

72

In [18]:
# olist_orders

# Fill Null values
df_olist_orders_dataset['order_delivered_carrier_date'] = df_olist_orders_dataset['order_delivered_carrier_date'].fillna('1900-01-01 00:00:00')
df_olist_orders_dataset['order_delivered_customer_date'] = df_olist_orders_dataset['order_delivered_customer_date'].fillna('1900-01-01 00:00:00')

# Convert columns to datetime
df_olist_orders_dataset['order_estimated_delivery_date'] = pd.to_datetime(df_olist_orders_dataset['order_estimated_delivery_date'])
df_olist_orders_dataset['order_purchase_timestamp'] = pd.to_datetime(df_olist_orders_dataset['order_purchase_timestamp'])


In [ ]:
# 4.2 Creating Calculated Columns
# 4.2.1 Create the following calculated columns in the order items dataset:
# 4.2.1.1 Total Price: Sum of product price and freight value

# olist_order_items

# Total Price: Sum of product price and freight value.
df_olist_order_items_dataset['total_price'] = df_olist_order_items_dataset['price'] + df_olist_order_items_dataset['freight_value']

#4.2.1.2 Delivery Time: Difference between the delivery date and the order purchase date. 

# olist_orders

# Delivery Time: Difference between the delivery date and the order purchase date.
df_olist_orders_dataset['delivery_time'] = df_olist_orders_dataset['order_estimated_delivery_date'] - df_olist_orders_dataset['order_purchase_timestamp']

df_olist_orders_dataset['delivery_time'] = df_olist_orders_dataset['delivery_time'].astype(str)
df_olist_orders_dataset.to_sql("olist_orders", con=arg_eng_olist, if_exists="replace", index=False)

# 4.2.1.3 Payment Count: Sum of payment installments for each order. 


# olist_order_payments

#Optional:
# Drop all duplicate payment_count columns if they exist
df_olist_order_payments_dataset = df_olist_order_payments_dataset.drop(
    columns=[col for col in df_olist_order_payments_dataset.columns if col.startswith('payment_count')],
    errors='ignore'
)

# Create the payment_count DataFrame
payment_count_df = df_olist_order_payments_dataset.groupby('order_id', as_index=False).agg(
    payment_count=('payment_installments', 'sum')
)

# Merge the new payment_count column
df_olist_order_payments_dataset = df_olist_order_payments_dataset.merge(
    payment_count_df, on='order_id', how='left'
)

#Save the updated dataframe to SQL
df_olist_order_payments_dataset.to_sql("olist_order_payments", con=arg_eng_olist, if_exists="replace", index=False)


# Display the result
print(df_olist_order_payments_dataset.head())

# 4.2.1.4 Profit Margin: Subtract freight value from product price to calculate a rough profit 
# estimate. 

# olist_order_items

# Profit Margin: Subtract freight value from product price to calculate a rough profit estimate.

df_olist_order_items_dataset['profit_margin'] = df_olist_order_items_dataset['price'] - df_olist_order_items_dataset['freight_value']

df_olist_order_items_dataset.to_sql("olist_order_items", con=arg_eng_olist, if_exists="replace", index=False)



In [ ]:
# olist_customers

df_olist_customers_dataset.to_sql("olist_customers", con=arg_eng_olist, if_exists="replace", index=False)

# olist_geolocation

df_olist_geolocation_dataset.to_sql("olist_geolocation", con=arg_eng_olist, if_exists="replace", index=False)

# olist_order_reviews

df_olist_order_reviews_dataset.to_sql("olist_order_reviews", con=arg_eng_olist, if_exists="replace", index=False)